# ARFIMA Modelling Guide

This notebook demonstrates how to:
- Generate ARFIMA time series
- Estimate model order (p, d, q)
- Fit the custom ARFIMA model
- Inspect diagnostics
- Forecast future values
- Illustrate fractional differencing


In [ ]:
# Imports and setup
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Add src to path
sys.path.append(os.path.join(os.getcwd(), 'src'))

from analysis.arfima_modelling import ARFIMAModel, estimate_arfima_order, arfima_simulation

plt.style.use('seaborn-v0_8')
sns.set_palette('husl')
os.makedirs('results/figures', exist_ok=True)
os.makedirs('results/tables', exist_ok=True)


In [ ]:
# 1) Simulate ARFIMA time series and save
np.random.seed(42)
true_d = 0.3
true_ar = np.array([0.5])
true_ma = np.array([0.3])

series = arfima_simulation(
    n=1000,
    d=true_d,
    ar_params=true_ar,
    ma_params=true_ma,
    sigma=1.0,
    seed=42
)

print(f"Generated series length: {len(series)}")
print(f"True params: d={true_d}, AR={true_ar.tolist()}, MA={true_ma.tolist()}")

pd.DataFrame({'y': series}).to_csv('results/tables/arfima_simulated.csv', index=False)
plt.figure(figsize=(12,4))
plt.plot(series[:300], lw=1)
plt.title('Simulated ARFIMA series (first 300)')
plt.xlabel('t'); plt.ylabel('y'); plt.grid(True, alpha=0.3)
plt.show()


In [ ]:
# 2) Estimate model order (p, d, q)
print('Estimating order...')
p_est, d_est, q_est = estimate_arfima_order(series, max_p=2, max_q=2, d_values=[0.1,0.2,0.3,0.4])
print(f"Estimated order: ARFIMA({p_est},{d_est:.3f},{q_est})")


In [ ]:
# 3) Fit the ARFIMA model and view summary
model = ARFIMAModel(p=p_est, d=d_est, q=q_est)
model.fit(series)
summary = model.summary()
summary


In [ ]:
# 4) Diagnostics
model.plot_diagnostics()


In [ ]:
# 5) Forecast next 50 steps and plot with confidence intervals
steps = 50
fc, lo, hi = model.forecast(steps=steps, alpha=0.05, interval_method='bootstrap', B=300, seed=42)

plt.figure(figsize=(12,5))
hist = series[-200:]
plt.plot(hist, label='History')
# extend x for forecast
x_fc = np.arange(len(series), len(series)+steps)
plt.plot(x_fc, fc, '--r', label='Forecast (mean)')
plt.fill_between(x_fc, lo, hi, color='r', alpha=0.2, label='95% CI')
plt.title('Forecast (50 steps) with 95% Confidence Interval')
plt.xlabel('t'); plt.ylabel('y'); plt.legend(); plt.grid(True, alpha=0.3)
plt.savefig('results/figures/arfima_forecast.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# 6) Fractional differencing illustration
probe_d = [0.0, 0.2, 0.3, 0.4]
plt.figure(figsize=(12,6))
for i, d in enumerate(probe_d, start=1):
    plt.subplot(2,2,i)
    diffed = model._fractional_difference(series, d)
    plt.plot(diffed[:200])
    plt.title(f'd={d}')
    plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('results/figures/fracdiff_examples.png', dpi=300, bbox_inches='tight')
plt.show()
